# Changes in aerobic habitat

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cartopy
import cartopy.crs as ccrs
import cmocean
import data_collections as dc
import funnel
import intake
import matplotlib.pyplot as plt
import numpy as np
import util
import xarray as xr

/glade/work/mclong/miniconda3/envs/metabolic/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
try:
    cluster
    client
except:
    cluster, client = util.get_ClusterClient(memory='25GB')
    cluster.scale(6)
client

/glade/work/mclong/miniconda3/envs/metabolic/lib/python3.7/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38118 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/38118/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/38118/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.205.22:41264,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/38118/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
catalog_json_file = funnel.to_intake_esm(agg_member_id=True)

sub_spec = dict(
    name='drift-corrected',
    experiment=['20C', 'RCP85'],
    member_id=dc.ocean_bgc_member_ids,
)

catalog = funnel.to_intake_esm(agg_member_id=True).search(**sub_spec)
catalog

,unique
experiment,2
component,1
stream,1
member_id,32
variable,8
name,1
path,464


In [5]:
def fix_dataset(ds):
    ds['depth_habitat_trait_wgt'] = ds.depth_habitat_trait_wgt.where(ds.KMT > 0)
    return ds.set_coords(['TLAT', 'TLONG'])


cat = catalog.search(variable='depth_habitat_trait_wgt')
dsets = cat.to_dataset_dict(preprocess=fix_dataset, zarr_kwargs={'use_cftime': True})
dsets = {k: ds.drop(['Ac', 'Eo']) for k, ds in dsets.items()}


exp_keys = ['20C.ocn.pop.h.drift-corrected', 'RCP85.ocn.pop.h.drift-corrected']
ds = xr.concat([dsets[k] for k in exp_keys], dim='time', coords='minimal')
ds['TLAT'] = ds.TLAT[0, :, :]
ds['TLONG'] = ds.TLONG[0, :, :]
ds


--> The keys in the returned dictionary of datasets are constructed as follows:
	'experiment.component.stream.name'


ValueError: One or more of the specified variables cannot be found in this dataset

In [ ]:
ds.depth_habitat_trait_wgt

In [ ]:
yrfrac = util.year_frac(ds.time)
tndx_ref = np.where(yrfrac < 1966)[0]
tndx_2100 = np.where(yrfrac > 2080)[0]

In [ ]:
with xr.set_options(keep_attrs=True):
    aero_hab_glb = (ds.depth_habitat_trait_wgt * ds.TAREA).sum(['nlat', 'nlon']).compute()
    aero_hab_glb *= 1e-6 * 1e-6
    aero_hab_glb.attrs['units'] = '10$^6$ m$^3$'
    aero_hab_glb.attrs['long_name'] = 'Trait-space weighted aerobic habitat'

    aero_hab_glb_control = aero_hab_glb.isel(time=tndx_ref).mean(['time', 'member_id'])
    aero_hab_glb_normalized = (
        100.0 * (aero_hab_glb - aero_hab_glb_control) / aero_hab_glb_control
    ).compute()
    aero_hab_glb_normalized['units'] = '%'


aero_hab_glb

In [ ]:
fig, ax = plt.subplots()

for member_id in aero_hab_glb.member_id.values:
    aero_hab_glb.sel(member_id=member_id).plot(ax=ax)

ax.set_title('Global ocean aerobic habitat volume');

In [ ]:
fig, ax = plt.subplots()

for member_id in aero_hab_glb.member_id.values:
    ax.plot(
        yrfrac,
        aero_hab_glb_normalized.sel(member_id=member_id),
        linestyle='-',
        color='gray',
        linewidth=0.5,
    )

ax.plot(yrfrac, aero_hab_glb_normalized.mean('member_id'), '-', color='k', linewidth=2)
ax.axhline(0.0, linewidth=0.5, color='k')
ax.set_title('Change in global ocean aerobic habitat');

In [ ]:
ds.depth_habitat_trait_wgt.isel(member_id=0, time=0).plot()

In [ ]:
habitat_contraction = (
    ds.depth_habitat_trait_wgt.isel(time=tndx_2100).mean(['time', 'member_id'])
    - ds.depth_habitat_trait_wgt.isel(time=tndx_ref).mean(['time', 'member_id'])
).compute()
habitat_contraction /= 1000e2
habitat_contraction *= 100.0
habitat_contraction.attrs['long_name'] = 'Habitat change'
habitat_contraction.attrs['units'] = '%'
habitat_contraction.name = 'habitat_contraction'

habitat_contraction

In [ ]:
habitat_contraction.plot()

In [ ]:
ds_plot = util.pop_add_cyclic(habitat_contraction.to_dataset())
ds_plot

In [ ]:
prj = ccrs.Robinson(central_longitude=305.0)
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw=dict(projection=prj))


cmap = cmocean.cm.curl_r

pc = ax.contourf(
    ds_plot.TLONG,
    ds_plot.TLAT,
    ds_plot.habitat_contraction,
    levels=np.arange(-40, 42, 2),
    cmap=cmap,
    transform=ccrs.PlateCarree(),
)

land = ax.add_feature(
    cartopy.feature.NaturalEarthFeature(
        'physical', 'land', '110m', edgecolor='face', facecolor='lightgray'
    )
)
plt.colorbar(pc, shrink=0.5);